In [1]:
from contrans import contrans
import numpy as np
import pandas as pd
import dotenv
import json
import os
import requests

dotenv.load_dotenv()


True

In [2]:
ct = contrans()
ct.mypassword
ct.congresskey

'TEpTHpL0UEqkAiMhHerXxdMW9or5xGze4urGA00R'

In [3]:
goodbioguide = ct.get_bioguide(name='good', state='Virginia', district=5)
goodbioguide

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,terms.item
0,G000595,5.0,"good, bob",Republican,Virginia,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/G000595?for...,Image courtesy of the Member,https://www.congress.gov/img/member/g000595_20...,"[{'chamber': 'House of Representatives', 'star..."
1,G000280,5.0,"goode, virgil h., jr.",Republican,Virginia,2022-12-16T21:56:51Z,https://api.congress.gov/v3/member/G000280?for...,"Image, Congressional Pictorial Directory, 109th.",https://www.congress.gov/img/member/g000280_20...,"[{'chamber': 'House of Representatives', 'endY..."


In [4]:
goodid = goodbioguide.reset_index()['bioguideId'][0]
goodid

'G000595'

In [5]:
goodbills = ct.get_sponsoredlegislation(goodid)

In [ ]:
billurl = goodbills[0]['url']

In [7]:
ct.get_billdata(billurl)

{'bill': {'actions': {'count': 3,
   'url': 'https://api.congress.gov/v3/bill/118/hres/1509/actions?format=json'},
  'committees': {'count': 1,
   'url': 'https://api.congress.gov/v3/bill/118/hres/1509/committees?format=json'},
  'congress': 118,
  'cosponsors': {'count': 30,
   'countIncludingWithdrawnCosponsors': 30,
   'url': 'https://api.congress.gov/v3/bill/118/hres/1509/cosponsors?format=json'},
  'introducedDate': '2024-09-25',
  'latestAction': {'actionDate': '2024-09-25',
   'text': 'Referred to the House Committee on Energy and Commerce.'},
  'number': '1509',
  'originChamber': 'House',
  'originChamberCode': 'H',
  'sponsors': [{'bioguideId': 'G000595',
    'district': 5,
    'firstName': 'Bob',
    'fullName': 'Rep. Good, Bob [R-VA-5]',
    'isByRequest': 'N',
    'lastName': 'Good',
    'party': 'R',
    'state': 'VA',
    'url': 'https://api.congress.gov/v3/member/G000595?format=json'}],
  'textVersions': {'count': 1,
   'url': 'https://api.congress.gov/v3/bill/118/hres/

In [7]:
import requests
r = requests.get(goodbills[0]['url'],
                 params={'api_key': ct.congresskey})
texturl = json.loads(r.text)['bill']['textVersions']['url']
r = requests.get(texturl,
                 params={'api_key': ct.congresskey})
toscrape = json.loads(r.text)['textVersions'][0]['formats'][0]['url']

In [9]:
from bs4 import BeautifulSoup

r = requests.get(toscrape)
mysoup = BeautifulSoup(r.text, 'html.parser')
billtext =  mysoup.text

<html><body><pre>
[Congressional Bills 118th Congress]
[From the U.S. Government Publishing Office]
[H. Res. 1509 Introduced in House (IH)]

&lt;DOC&gt;






118th CONGRESS
  2d Session
H. RES. 1509

Supporting the designation of the week beginning November 11, 2024, as 
  ``National Pregnancy Center Week'' to recognize the vital role that 
community-supported pregnancy centers play in saving lives and serving 
        women and men faced with difficult pregnancy decisions.


_______________________________________________________________________


                    IN THE HOUSE OF REPRESENTATIVES

                           September 25, 2024

 Mr. Good of Virginia (for himself, Mr. Clyde, Mr. Mooney, Mr. Gosar, 
 Mr. Posey, Mr. Burgess, Mr. Bacon, Mr. Fleischmann, Mr. Grothman, Mr. 
Rose, Mr. Fulcher, Mr. Biggs, Mr. Higgins of Louisiana, Mr. Ogles, Mr. 
 Kelly of Pennsylvania, Mr. Lamborn, Mrs. Harshbarger, Mr. Norman, Mr. 
  Wilson of South Carolina, Mrs. Miller of Illinois, Mr. 

In [11]:
mysoup.find_all('pre')

[<pre>
 [Congressional Bills 118th Congress]
 [From the U.S. Government Publishing Office]
 [H. Res. 1509 Introduced in House (IH)]
 
 &lt;DOC&gt;
 
 
 
 
 
 
 118th CONGRESS
   2d Session
 H. RES. 1509
 
 Supporting the designation of the week beginning November 11, 2024, as 
   ``National Pregnancy Center Week'' to recognize the vital role that 
 community-supported pregnancy centers play in saving lives and serving 
         women and men faced with difficult pregnancy decisions.
 
 
 _______________________________________________________________________
 
 
                     IN THE HOUSE OF REPRESENTATIVES
 
                            September 25, 2024
 
  Mr. Good of Virginia (for himself, Mr. Clyde, Mr. Mooney, Mr. Gosar, 
  Mr. Posey, Mr. Burgess, Mr. Bacon, Mr. Fleischmann, Mr. Grothman, Mr. 
 Rose, Mr. Fulcher, Mr. Biggs, Mr. Higgins of Louisiana, Mr. Ogles, Mr. 
  Kelly of Pennsylvania, Mr. Lamborn, Mrs. Harshbarger, Mr. Norman, Mr. 
   Wilson of South Carolina, Mrs. Mi

In [6]:
import requests

# sponsored legislation
root = 'https://api.congress.gov/v3/'
endpoint = f'/member/{goodid}/sponsored-legislation'
headers = ct.make_headers()
params = {'api_key': ct.congresskey,
          'limit': 250}
r = requests.get(root + endpoint,
                 headers=headers,
                 params=params)
r

<Response [200]>

In [10]:
json.loads(r.text)['sponsoredLegislation'][2]

{'congress': 118,
 'introducedDate': '2024-05-23',
 'latestAction': {'actionDate': '2024-07-05',
  'text': 'Placed on the Union Calendar, Calendar No. 474.'},
 'number': '8534',
 'policyArea': {'name': None},
 'title': 'Protecting Student Athletes’ Economic Freedom Act of 2024',
 'type': 'HR',
 'url': 'https://api.congress.gov/v3/bill/118/hr/8534?format=json'}

In [3]:
useragent = ct.get_useragent()
useragent

'python-requests/2.32.3'

In [4]:
headers = ct.make_headers()
headers

{'User-Agent': 'python-requests/2.32.3', 'From': 'fvq3xv@virginia.edu'}

In [5]:
members_df = ct.get_bioguideIDs()

In [10]:
members_df

,bioguideId,district,name,partyName,state,updateDate,url,terms.item,depiction.attribution,depiction.imageUrl
0,M001229,10.0,"McIver, LaMonica",Democratic,New Jersey,2024-09-24T20:50:13Z,https://api.congress.gov/v3/member/M001229?for...,"[{'chamber': 'House of Representatives', 'star...",NaN,NaN
1,E000236,NaN,"Evans, Daniel J.",Republican,Washington,2024-09-23T14:08:49Z,https://api.congress.gov/v3/member/E000236?for...,"[{'chamber': 'Senate', 'endYear': 1989, 'start...","<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/e000236_20...
2,S000068,NaN,"Sasser, Jim",Democratic,Tennessee,2024-09-12T19:33:39Z,https://api.congress.gov/v3/member/S000068?for...,"[{'chamber': 'Senate', 'endYear': 1995, 'start...","<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/s000068_20...
3,H001097,NaN,"Helmy, George S.",Democratic,New Jersey,2024-09-12T12:42:19Z,https://api.congress.gov/v3/member/H001097?for...,"[{'chamber': 'Senate', 'startYear': 2024}]",NaN,NaN
4,R000599,25.0,"Ruiz, Raul",Democratic,California,2024-09-11T16:29:00Z,https://api.congress.gov/v3/member/R000599?for...,"[{'chamber': 'House of Representatives', 'star...",Image courtesy of the Member,https://www.congress.gov/img/member/66e1aec832...
...,...,...,...,...,...,...,...,...,...,...
19,F000229,6.0,"Flynt, John J., Jr.",Democratic,Georgia,2022-12-16T21:56:51Z,https://api.congress.gov/v3/member/F000229?for...,"[{'chamber': 'House of Representatives', 'endY...",Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/f000229_20...
20,F000218,7.0,"Flowers, Walter",Democratic,Alabama,2022-12-16T21:56:51Z,https://api.congress.gov/v3/member/F000218?for...,"[{'chamber': 'House of Representatives', 'endY...",Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/f000218_20...
21,F000215,1.0,"Florio, James J.",Democratic,New Jersey,2022-12-16T21:56:51Z,https://api.congress.gov/v3/member/F000215?for...,"[{'chamber': 'House of Representatives', 'endY...",Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/f000215_20...
22,F000209,11.0,"Flood, Daniel J.",Democratic,Pennsylvania,2022-12-16T21:56:51Z,https://api.congress.gov/v3/member/F000209?for...,"[{'chamber': 'House of Representatives', 'endY...",Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/f000209_20...


In [7]:
ideo = ct.get_ideology()
members = ideo[['bioname', 'icpsr', 'party_code']]
members

,bioname,icpsr,party_code
0,"ROGERS, Mike Dennis",20301,200
1,"SEWELL, Terri",21102,100
2,"PALMER, Gary James",21500,200
3,"CARL, Jerry L.",22108,200
4,"MOORE, Barry",22140,200
...,...,...,...
443,"STEIL, Bryan",21970,200
444,"TIFFANY, Thomas P.",21989,200
445,"FITZGERALD, Scott",22115,200
446,"VAN ORDEN, Derrick",22370,200


In [8]:
votes = ct.get_votes()
votes

,congress,chamber,rollnumber,icpsr,cast_code,prob
0,118,House,1,14854,6,100.0
1,118,House,1,14863,6,98.2
2,118,House,1,14873,1,100.0
3,118,House,1,15029,1,100.0
4,118,House,1,15433,1,100.0
...,...,...,...,...,...,...
486659,118,House,1118,31101,6,100.0
486660,118,House,1118,31102,9,99.6
486661,118,House,1118,39301,6,100.0
486662,118,House,1118,39307,6,100.0


In [ ]:
house = house[['rollnumber', 'icpsr', 'cast_code']]

In [ ]:
house_mat = pd.merge(house, house, on='rollnumber')
house_mat

In [ ]:
house_mat = pd.merge(house, house, on='rollnumber')
house_mat['agree'] = house_mat['cast_code_x'] == house_mat['cast_code_y']
house_mat = house_mat.groupby(['icpsr_x', 'icpsr_y'])['agree'].agg({'agree': 'mean'})
house_mat = house_mat.reset_index()

In [ ]:
agreement = pd.merge(house_mat, members,
                     left_on = 'icpsr_x',
                     right_on = 'icpsr')
agreement = agreement.rename({'bioname':'member1',
                              'party_code':'party1'}, axis=1)
agreement = agreement.drop(['icpsr_x'], axis=1)
agreement = pd.merge(agreement, members,
                     left_on = 'icpsr_y',
                     right_on = 'icpsr')
agreement = agreement.rename({'bioname':'member2',
                              'party_code':'party2'}, axis=1)
agreement = agreement[['member1', 'party1', 'member2', 'party2', 'agree']]
agreement


In [ ]:
agreement[agreement['member1'].str.contains('GOOD,')].sort_values('agree',  ascending=False).head(30)